# CASE: Busca Semântica + PGvector

**Autor:** Gustavo das Neves Ubeda  
**Contato:** [gustavo.n.ubeda@gmail.com.br](mailto:gustavo.n.ubeda@gmail.com.br)

---

### Instalação do Docker

Para instalar e rodar o Docker, consulte a documentação: [Docker Desktop](https://www.docker.com/products/docker-desktop/).

---

### Instalação do Postgre com PGvector

Para instalar e rodar o PostgreSQL com a extensão PGvector, siga os passos abaixo:

1. Consulte a documentação: [PGvector no GitHub](https://github.com/pgvector/pgvector#getting-started).
2. Baixe a imagem Docker:
   ```bash
   docker pull pgvector/pgvector:pg17
   ```
3. Execute o container com a imagem:
   ```bash
   docker run -d --name pgvector -p 5432:5432 -v pg-data:/home/postgres/pgdata/data -e POSTGRES_PASSWORD=password pgvector/pgvector:pg17
   ```

---

### Instalação do Ollama

Para instalar e rodar o Ollama, siga os passos abaixo:

1. Consulte a documentação: [Ollama no GitHub](https://github.com/ollama/ollama/blob/main/docs/api.md).
2. Baixe e execute a imagem Docker:
   ```bash
   docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
   ```
3. Execute os modelos de IA disponíveis:
   - **Mistral:**
     ```bash
     docker exec -it ollama ollama run mistral
     ```
   - **Llama3:**
     ```bash
     docker exec -it ollama ollama run llama3
     ```
   - **Nomic Embed Text:**
     ```bash
     docker exec -it ollama ollama run nomic-embed-text
     ```

### Instalação e execusão dos containers

In [ ]:
!docker pull pgvector/pgvector:pg17
!docker run -d --name pgvector -p 5432:5432 -v pg-data:/home/postgres/pgdata/data -e POSTGRES_PASSWORD=password pgvector/pgvector:pg17
!docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
!docker exec ollama ollama run nomic-embed-text

### Instalação de libs adicionais

In [ ]:
!pip install tqdm
!pip install pandas
!pip install psycopg2
!pip install requests
# !pip install SQLAlchemy

In [36]:
import requests
import psycopg2
import pandas as pd
from tqdm.notebook import tqdm as tqdm_notebook

# import sqlalchemy as sa

### Parâmetros de configuração

In [37]:
# Ollama API
ollama_host = "http://host.docker.internal:11434"
model_use = "nomic-embed-text" # mistral, llama3, nomic-embed-text

# Postgres +  PGvector
dbname ='postgres'
user = 'postgres'
password = 'password'
host = 'localhost'
port = '5432'

### Implantação das funções

In [38]:
# Abstração para excução de querys no Postgres
def exe_query(query):
    conn = psycopg2.connect(dbname=dbname,user=user,password=password,host=host,port=port)
    cur = conn.cursor()
    cur.execute(query)

    try:
        data = cur.fetchall()
        cols = [desc[0] for desc in cur.description]

        conn.commit()
        cur.close()
        conn.close()

        return pd.DataFrame(data, columns=cols)
    except:
        conn.commit()
        cur.close()
        conn.close()

        return []

In [39]:
# Busca os modelos disponíveis no Ollama local
def get_models():
    response = requests.get(f"{ollama_host}/api/tags")
    return response.json()

In [40]:
# Gera o embedding para o texto a partir do modelo selecionado
def create_embed(text, model=model_use):
    data = {
        "model": model,
        "input": text
    }
    response = requests.post(f"{ollama_host}/api/embed", json=data)

    try:
        return response.json()["embeddings"][0]
    except:
        print(response.json())

### Ingerindo tabela de produtos + Embeddings

In [41]:
df = pd.read_excel("./tabela_produto.xls")
df.columns

Index(['id_produto', 'nome_produto', 'laboratorio', 'substancia',
       'apresentacao', 'classe_terapeutica', 'tipo_produto', 'tarja'],
      dtype='object')

In [42]:
# Eliminando ids duplicados da base
df = df.drop_duplicates(subset="id_produto")
datas = df.to_dict(orient="records")
datas[0]

{'id_produto': 7891106000956,
 'nome_produto': 'BAYCUTEN N',
 'laboratorio': 'BAYER S.A.',
 'substancia': '21-ACETATO DE DEXAMETASONA;CLOTRIMAZOL',
 'apresentacao': '10 MG/G + 0,443 MG/G CREM DERM CT BG AL X 40 G',
 'classe_terapeutica': 'D7B2 - CORTICOESTERÓIDES ASSOCIADOS A ANTIMICOTICOS',
 'tipo_produto': 'Novo',
 'tarja': '- (*) '}

##### Gerando os embedings

In [43]:
# Estratégia para organização do texto que será vetorizado
texts_to_embeds = [
    f"{i['nome_produto']} ({i['apresentacao']}) - {i['tarja']} - Contém: {i['substancia']}" 
    for i in datas
]

print("Exemplo:", texts_to_embeds[0])

Exemplo: BAYCUTEN N (10 MG/G + 0,443 MG/G CREM DERM CT BG AL X 40 G) - - (*)  - Contém: 21-ACETATO DE DEXAMETASONA;CLOTRIMAZOL


In [44]:
get_models()

{'models': [{'name': 'nomic-embed-text:latest',
   'model': 'nomic-embed-text:latest',
   'modified_at': '2025-01-15T19:36:17.546777188Z',
   'size': 274302450,
   'digest': '0a109f422b47e3a30ba2b10eca18548e944e8a23073ee3f3e947efcf3c45e59f',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'nomic-bert',
    'families': ['nomic-bert'],
    'parameter_size': '137M',
    'quantization_level': 'F16'}}]}

In [45]:
# Criando os embeddings
embeds = []
for tx in tqdm_notebook(texts_to_embeds):
    eb = create_embed(tx, model=model_use)
    embeds.append(eb)

len_vec = len(embeds[0])

df["embedding"] = [str(i) for i in embeds]
df.to_csv("./tabela_produto_backup.csv")

print("Dimensão dos vetores:", len_vec)

  0%|          | 0/996 [00:00<?, ?it/s]

Dimensão dos vetores: 768


##### Salvando tabela vetorizada no Postgres

In [48]:
# Ativando extenção do PGvector, Declarando e Criando a tabela no Postgres
exe_query(f"""
CREATE EXTENSION IF NOT EXISTS vector;

DROP TABLE IF EXISTS produtos CASCADE;
          
CREATE TABLE IF NOT EXISTS produtos (
    id_produto BIGINT PRIMARY KEY,
    nome_produto VARCHAR(512),
    laboratorio VARCHAR(512),
    substancia VARCHAR(512),
    apresentacao VARCHAR(512),
    classe_terapeutica VARCHAR(512),
    tipo_produto VARCHAR(512),
    tarja VARCHAR(512),
    embedding VECTOR({len_vec})
);
""")

[]

In [49]:
# Inserindo dados no Postgress
datas = df.to_dict(orient="records")
for data in tqdm_notebook(datas):
    cols = ", ".join([i for i in data])
    vals = tuple([data[i] for i in data])

    qr = f"INSERT INTO produtos ({cols}) VALUES {vals}"
    
    try:
        exe_query(qr)
    except:
        print(f"ERROR: {qr}")

  0%|          | 0/996 [00:00<?, ?it/s]

### Configurando Upsert para Tabela

In [50]:
# Função de inclusão/Atualização de produtos na tabela do Postgress
def upsert_data(data):
    text_to_embed = f"{data['nome_produto']} ({data['apresentacao']}) - {data['tarja']} - Contém: {data['substancia']}"
    embed = create_embed(text_to_embed, model=model_use)
    data["embedding"] = str(embed)
    
    cols = ", ".join([i for i in data])
    vals = tuple([data[i] for i in data])

    qr = f"""
    INSERT INTO produtos ({cols}) VALUES {vals}
    ON CONFLICT (id_produto)
    DO UPDATE SET
        nome_produto = EXCLUDED.nome_produto,
        laboratorio = EXCLUDED.laboratorio,
        substancia = EXCLUDED.substancia,
        apresentacao = EXCLUDED.apresentacao,
        classe_terapeutica = EXCLUDED.classe_terapeutica,
        tipo_produto = EXCLUDED.tipo_produto,
        tarja = EXCLUDED.tarja,
        embedding = EXCLUDED.embedding;
    """

    exe_query(qr)
    return exe_query(f"SELECT * FROM produtos WHERE id_produto = {data['id_produto']}")

In [51]:
# Exemplo de mudança de dados num produto
data = {
    'id_produto': 7896862993009,
    'nome_produto': 'DALMADORMI',
    'laboratorio': 'MEDQUIMICA INDUSTRIA FARMACEUTICA LTDA.',
    'substancia': 'CLORIDRATO DE FLURAZEPAM',
    'apresentacao': '30 MG COM REV CT BL AL PLAS TRANS X 30',
    'classe_terapeutica': 'N5B1 - HIPNÓTICOS E SEDATIVOS NÃO BARBITÚRICOS PUROS',
    'tipo_produto': 'Similar',
    'tarja': 'Tarja Preta'
}

upsert_data(data)

,id_produto,nome_produto,laboratorio,substancia,apresentacao,classe_terapeutica,tipo_produto,tarja,embedding
0,7896862993009,DALMADORMI,MEDQUIMICA INDUSTRIA FARMACEUTICA LTDA.,CLORIDRATO DE FLURAZEPAM,30 MG COM REV CT BL AL PLAS TRANS X 30,N5B1 - HIPNÓTICOS E SEDATIVOS NÃO BARBITÚRICOS...,Similar,Tarja Preta,"[0.038446784,0.05230631,-0.15728274,-0.0074414..."


### Busca semântica (PGvector + Ollama)

In [52]:
# Busca semântica por coseno sentre vetores
# Obs: Produto interno (<#>), distância cosseno (<=>), distância L2 (<->) e distância L1 (<+>
def busca_semantica(tx, n_itens=5):
    embed = create_embed(tx, model=model_use)

    qr = f"SELECT * FROM produtos ORDER BY embedding <=> '{embed}' LIMIT {n_itens};"
    return exe_query(qr)

##### Testes de busca

In [53]:
busca_semantica("amoxicilina", n_itens=5)

,id_produto,nome_produto,laboratorio,substancia,apresentacao,classe_terapeutica,tipo_produto,tarja,embedding
0,7898148298884,AMOXICILINA,PRATI DONADUZZI & CIA LTDA,AMOXICILINA,50 MG/ML PO SUS OR CX 50 FR VD AMB X 150 ML + ...,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.06628664,0.04506001,-0.16958027,0.024697265..."
1,7898148298891,AMOXICILINA,PRATI DONADUZZI & CIA LTDA,AMOXICILINA,50 MG/ML PO SUS OR CT FR VD AMB X 60 ML\xa0+ COP,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.06777089,0.043739434,-0.17210077,0.02339835..."
2,7896181918363,AMOXICILINA,ACHE LABORATORIOS FARMACEUTICOS SA,AMOXICILINA TRI-HIDRATADA,50 MG/ML PO SUS OR CT FR VD AMB X 150 ML + SER...,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.07063644,0.049844872,-0.17516832,0.02114205..."
3,7891317001650,AMOXICILINA,EUROFARMA LABORATÓRIOS S.A.,AMOXICILINA;AMOXICILINA TRI-HIDRATADA,100 MG/ML PO SUS OR CT FR VD AMB X 150 ML + COP,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.06040595,0.04981944,-0.17900415,0.030900277..."
4,7896523227030,AMOXICILINA,CIMED INDUSTRIA S.A.,AMOXICILINA TRIHIDRATADA,50 MG/ML PO SUS CX 50 FR PLAS OPC X 60 ML + 5...,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.07388419,0.0434783,-0.17170998,0.010910696,..."


In [54]:
busca_semantica("amoxicilina 500m", n_itens=5)

,id_produto,nome_produto,laboratorio,substancia,apresentacao,classe_terapeutica,tipo_produto,tarja,embedding
0,7898148298884,AMOXICILINA,PRATI DONADUZZI & CIA LTDA,AMOXICILINA,50 MG/ML PO SUS OR CX 50 FR VD AMB X 150 ML + ...,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.06628664,0.04506001,-0.16958027,0.024697265..."
1,7898148298891,AMOXICILINA,PRATI DONADUZZI & CIA LTDA,AMOXICILINA,50 MG/ML PO SUS OR CT FR VD AMB X 60 ML\xa0+ COP,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.06777089,0.043739434,-0.17210077,0.02339835..."
2,7898093850175,IQUEGO-AMOXICILINA,INDÚSTRIA QUÍMICA DO ESTADO DE GOIÁS S/A - IQUEGO,AMOXICILINA TRIHIDRATADA,500 MG CAP DURA CX BL AL PVC TRANS X 500,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Similar,- (*),"[0.034949344,0.044494662,-0.18152067,-0.009925..."
3,7897076906007,AMOXICILINA,RANBAXY FARMACÊUTICA LTDA,AMOXICILINA,500 MG CAP DURA CT BL AL PLAS PVC/PVDC TRANS X 6,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.04319027,0.04094665,-0.16153677,0.006073045..."
4,7898093850151,IQUEGO-AMOXICILINA,INDÚSTRIA QUÍMICA DO ESTADO DE GOIÁS S/A - IQUEGO,AMOXICILINA TRI-HIDRATADA,50 MG/ML PO SUS CX 50 FR VD AMB X 80 ML (60 ML...,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Similar,- (*),"[0.058411773,0.05524169,-0.17910674,0.01856944..."


In [55]:
busca_semantica("amoxicilina 500mg generico", n_itens=5)

,id_produto,nome_produto,laboratorio,substancia,apresentacao,classe_terapeutica,tipo_produto,tarja,embedding
0,7898093850151,IQUEGO-AMOXICILINA,INDÚSTRIA QUÍMICA DO ESTADO DE GOIÁS S/A - IQUEGO,AMOXICILINA TRI-HIDRATADA,50 MG/ML PO SUS CX 50 FR VD AMB X 80 ML (60 ML...,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Similar,- (*),"[0.058411773,0.05524169,-0.17910674,0.01856944..."
1,7897076906007,AMOXICILINA,RANBAXY FARMACÊUTICA LTDA,AMOXICILINA,500 MG CAP DURA CT BL AL PLAS PVC/PVDC TRANS X 6,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.04319027,0.04094665,-0.16153677,0.006073045..."
2,7898093850175,IQUEGO-AMOXICILINA,INDÚSTRIA QUÍMICA DO ESTADO DE GOIÁS S/A - IQUEGO,AMOXICILINA TRIHIDRATADA,500 MG CAP DURA CX BL AL PVC TRANS X 500,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Similar,- (*),"[0.034949344,0.044494662,-0.18152067,-0.009925..."
3,7898361880958,AMOXICILINA,AUROBINDO PHARMA INDÚSTRIA FARMACÊUTICA LIMITADA,AMOXICILINA TRIHIDRATADA,500 MG CAP DURA CT BL AL PLAS PVC TRANS X 350,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.048268866,0.038747996,-0.16368046,0.0041478..."
4,7898148298884,AMOXICILINA,PRATI DONADUZZI & CIA LTDA,AMOXICILINA,50 MG/ML PO SUS OR CX 50 FR VD AMB X 150 ML + ...,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,Tarja Vermelha sob restrição,"[0.06628664,0.04506001,-0.16958027,0.024697265..."


In [56]:
busca_semantica("genérico do FLUIMUCIL", n_itens=5)

,id_produto,nome_produto,laboratorio,substancia,apresentacao,classe_terapeutica,tipo_produto,tarja,embedding
0,7898074617438,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,40 MG/ML XPE CT FR VD AMB X 120ML + COP SBR MO...,R5C - EXPECTORANTES,Novo,- (*),"[0.0198729,0.043831605,-0.1829889,0.03138968,0..."
1,7898074617445,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,20 MG/ML XPE CT FR VD AMB X 120 ML + COP SBR F...,R5C - EXPECTORANTES,Novo,- (*),"[0.032021374,0.047884367,-0.17785738,0.0229062..."
2,7898074616714,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,40 MG/G GRAN SOL CT 16 ENV AL PE X 5 G\xa0,R5C - EXPECTORANTES,Novo,- (*),"[0.03179896,0.06094494,-0.17348383,-0.00330431..."
3,7895858017002,FLORINEFE,ASPEN PHARMA INDÚSTRIA FARMACÊUTICA LTDA,ACETATO DE FLUDROCORTISONA,"0,1 MG COM CT FR VD AMB X 100",H2A2 - CORTICOSTERÓIDES ORAIS PUROS,Novo,Tarja Vermelha,"[0.04250259,0.0430255,-0.18810739,-0.007496798..."
4,7898074617612,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,600 MG COM EFEV CT BL AL/AL X 16,R5C - EXPECTORANTES,Novo,- (*),"[0.043433454,0.05440264,-0.15940394,0.00229969..."


In [57]:
busca_semantica("contem a mesma composiçao do FLUIMUCIL", n_itens=5)

,id_produto,nome_produto,laboratorio,substancia,apresentacao,classe_terapeutica,tipo_produto,tarja,embedding
0,7898074617438,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,40 MG/ML XPE CT FR VD AMB X 120ML + COP SBR MO...,R5C - EXPECTORANTES,Novo,- (*),"[0.0198729,0.043831605,-0.1829889,0.03138968,0..."
1,7895858017002,FLORINEFE,ASPEN PHARMA INDÚSTRIA FARMACÊUTICA LTDA,ACETATO DE FLUDROCORTISONA,"0,1 MG COM CT FR VD AMB X 100",H2A2 - CORTICOSTERÓIDES ORAIS PUROS,Novo,Tarja Vermelha,"[0.04250259,0.0430255,-0.18810739,-0.007496798..."
2,7898074617445,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,20 MG/ML XPE CT FR VD AMB X 120 ML + COP SBR F...,R5C - EXPECTORANTES,Novo,- (*),"[0.032021374,0.047884367,-0.17785738,0.0229062..."
3,7898074616714,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,40 MG/G GRAN SOL CT 16 ENV AL PE X 5 G\xa0,R5C - EXPECTORANTES,Novo,- (*),"[0.03179896,0.06094494,-0.17348383,-0.00330431..."
4,7898074611276,FLUIMUCIL,ZAMBON LABORATÓRIOS FARMACÊUTICOS LTDA.,ACETILCISTEÍNA,600 MG COM EFEV CT BL AL/AL X 60,R5C - EXPECTORANTES,Novo,- (*),"[0.043017097,0.056886833,-0.15766566,0.0052714..."


In [62]:
busca_semantica("tarja preta", n_itens=5)

,id_produto,nome_produto,laboratorio,substancia,apresentacao,classe_terapeutica,tipo_produto,tarja,embedding
0,7895296280204,ALPRAZOLAM,MULTILAB INDUSTRIA E COMERCIO DE PRODUTOS FARM...,ALPRAZOLAM,2 MG COM CT BL AL PLAS TRANS X 30,N5C - TRANQUILIZANTES,Genérico,Tarja Preta,"[0.03712384,0.041326012,-0.12640926,-0.0043887..."
1,7891317010799,ALPRAZOLAM,EUROFARMA LABORATÓRIOS S.A.,ALPRAZOLAM,2 MG COM CT BL AL AL X 30,N5C - TRANQUILIZANTES,Genérico,Tarja Preta,"[0.040516324,0.04043837,-0.1296936,-0.00306286..."
2,7898361885496,ALPRAZOLAM,AUROBINDO PHARMA INDÚSTRIA FARMACÊUTICA LIMITADA,ALPRAZOLAM,"0,5 MG COM CT BL AL AL X 30",N5C - TRANQUILIZANTES,Genérico,Tarja Preta,"[0.050267003,0.04486824,-0.12595588,-0.0188857..."
3,7896004718804,ALPRAZOLAM,EMS S/A,ALPRAZOLAM,"0,5 MG COM CT BL AL PLAS TRANS X 30",N5C - TRANQUILIZANTES,Genérico,Tarja Preta,"[0.045229338,0.042533044,-0.12113424,-0.022081..."
4,7896112139034,ALPRAZOLAM,LABORATÓRIO TEUTO BRASILEIRO S/A,ALPRAZOLAM,"0,5 MG COM CT BL AL PLAS TRANS X 30",N5C - TRANQUILIZANTES,Genérico,Tarja Preta,"[0.045229338,0.042533044,-0.12113424,-0.022081..."
